In [16]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import scipy
from scipy import ndimage

from features import rgb2gray

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (30.0, 16.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
layer_name = 'MODIS_Aqua_CorrectedReflectance_TrueColor'
img_extension = ".jpg"

mask_name = 'MODIS_Aqua_Data_No_Data'
mask_extension = ".png"

data_dir = 'data/4326/'

labels_file = os.path.join(data_dir, layer_name + ".txt")

img_size = (8192, 4096)

def parse_labels(label_arr):
    """    
    None = Normal
    1 = missing data (expected)
    2 = missing data (unexpected)
    3 = miscoloration
    4 = edge warping
    5 = eclipse (missing data)
    6 = eclipse (only miscoloration)
    """
    label = 0
    if len(label_arr) == 0:
        label = 0
    else:
        if 1 in label_arr or 2 in label_arr or 5 in label_arr: # missing data labels!
            label = 1
        else:
            label = 0
    return label

# Check the labels file exists!
assert os.path.exists(labels_file), "Cannot find the {} file".format(layer_name + ".txt")
        
# Read in the file line by line
results_dict = {}
with open(labels_file) as f:
    file_lines = f.read().splitlines()
    num_total_img = len(file_lines)
    for line in file_lines:
        line_list = line.split()  
        split = line_list[0]
        datestring = line_list[1]
        label_arr = [int(item) for item in line_list[2:]]
        label = parse_labels(label_arr)
        
        # Construct and resize the image
        filename = os.path.join(data_dir, datestring, layer_name + img_extension)
        image = np.asarray(Image.open(filename).resize(img_size, Image.BILINEAR))
                
        # Grayscale the image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Threshold the image to black and white
        _, image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)
        
        # Get the mask image
        mask_filename = os.path.join(data_dir, datestring, mask_name + mask_extension)
        mask_img = np.asarray(Image.open(mask_filename).resize(img_size, Image.BILINEAR))
        
        # Grayscale and threshold to black and white
        mask_img = cv2.cvtColor(mask_img, cv2.COLOR_BGR2GRAY)
        _, mask_img = cv2.threshold(mask_img, 0, 255, cv2.THRESH_BINARY_INV)
        
        result = cv2.bitwise_or(image,image,mask = mask_img)
        print("{}: {}".format(datestring, np.sum(result - mask_img)))
        results_dict[datestring] = np.sum(result - mask_img)
print(results_dict)

2003-01-01: 49
2003-01-02: 23
2003-01-03: 53
2003-01-04: 55
2003-01-05: 69
2003-01-06: 20
2003-01-07: 39
2003-01-08: 53
2003-01-09: 53
2003-01-10: 50
2003-01-11: 56
2003-01-12: 72
2003-01-13: 22
2003-01-14: 62
2003-01-15: 71
2003-01-16: 73
2003-01-17: 117
2003-01-18: 58
2003-01-19: 144
2003-01-20: 113
2003-01-21: 186
2003-01-22: 70
2003-01-23: 27
2003-01-24: 102
2003-01-25: 83
2003-01-26: 207
2003-01-27: 72
2003-01-28: 96
2003-01-29: 21
2003-01-30: 54
2003-01-31: 27
2003-02-01: 95
2003-02-02: 91
2003-02-03: 67
2003-02-04: 4
2003-02-05: 9
2003-02-06: 83
2003-02-07: 56
2003-02-08: 95
2003-02-09: 39
2003-02-10: 43
2003-02-11: 17
2003-02-12: 19
2003-02-13: 85
2003-02-14: 58
2003-02-15: 522
2003-02-16: 30
2003-02-17: 170
2003-02-18: 20
2003-02-19: 13
2003-02-20: 67
2003-02-21: 255
2003-02-22: 87
2003-02-23: 84
2003-02-24: 23
2003-02-25: 53
2003-02-26: 32
2003-02-27: 45
2003-02-28: 23
2003-03-01: 47
2003-03-02: 28
2003-03-03: 14
2003-03-04: 52
2003-03-05: 24
2003-03-06: 44
2003-03-07: 15
200

2004-06-15: 223
2004-06-16: 162
2004-06-17: 130
2004-06-18: 203
2004-06-19: 295
2004-06-20: 232
2004-06-21: 208
2004-06-22: 136
2004-06-23: 228
2004-06-24: 189
2004-06-25: 265
2004-06-26: 203
2004-06-27: 262
2004-06-28: 238
2004-06-29: 216
2004-06-30: 316
2004-07-01: 330
2004-07-02: 271
2004-07-03: 105
2004-07-04: 139
2004-07-05: 382
2004-07-06: 248
2004-07-07: 143
2004-07-08: 160
2004-07-09: 236
2004-07-10: 302
2004-07-11: 274
2004-07-12: 190
2004-07-13: 228
2004-07-14: 265
2004-07-15: 149
2004-07-16: 143
2004-07-17: 151
2004-07-18: 225
2004-07-19: 148
2004-07-20: 284
2004-07-21: 132
2004-07-22: 157
2004-07-23: 173
2004-07-24: 190
2004-07-25: 107
2004-07-26: 129
2004-07-27: 151
2004-07-28: 234
2004-07-29: 276
2004-07-30: 224
2004-07-31: 122
2004-08-01: 104
2004-08-02: 176
2004-08-03: 207
2004-08-04: 183
2004-08-05: 356
2004-08-06: 248
2004-08-07: 206
2004-08-08: 179
2004-08-09: 132
2004-08-10: 346
2004-08-11: 282
2004-08-12: 230
2004-08-13: 181
2004-08-14: 230
2004-08-15: 179
2004-08-

2005-11-22: 44
2005-11-23: 202
2005-11-24: 45
2005-11-25: 6
2005-11-26: 57
2005-11-27: 14
2005-11-28: 81
2005-11-29: 34
2005-11-30: 100
2005-12-01: 53
2005-12-02: 15
2005-12-03: 71
2005-12-04: 22
2005-12-05: 30
2005-12-06: 105
2005-12-07: 77
2005-12-08: 92
2005-12-09: 65
2005-12-10: 25
2005-12-11: 109
2005-12-12: 80
2005-12-13: 70
2005-12-14: 112
2005-12-15: 6
2005-12-16: 49
2005-12-17: 110
2005-12-18: 61
2005-12-19: 56
2005-12-20: 51
2005-12-21: 232
2005-12-22: 48
2005-12-23: 87
2005-12-24: 41
2005-12-25: 63
2005-12-26: 64
2005-12-27: 27498285
2005-12-28: 27451069
2005-12-29: 27489602
2005-12-30: 27515805
2005-12-31: 27495070
2006-01-01: 52
2006-01-02: 23
2006-01-03: 94
2006-01-04: 39
2006-01-05: 29
2006-01-06: 40
2006-01-07: 16
2006-01-08: 15
2006-01-09: 39
2006-01-10: 17
2006-01-11: 98
2006-01-12: 18
2006-01-13: 22
2006-01-14: 48
2006-01-15: 137
2006-01-16: 50
2006-01-17: 50
2006-01-18: 72
2006-01-19: 75
2006-01-20: 93
2006-01-21: 21
2006-01-22: 50
2006-01-23: 56
2006-01-24: 16
2006

2007-05-07: 114
2007-05-08: 130
2007-05-09: 139
2007-05-10: 103
2007-05-11: 206
2007-05-12: 176
2007-05-13: 176
2007-05-14: 94
2007-05-15: 159
2007-05-16: 410
2007-05-17: 170
2007-05-18: 175
2007-05-19: 111
2007-05-20: 146
2007-05-21: 233
2007-05-22: 253
2007-05-23: 119
2007-05-24: 344
2007-05-25: 252
2007-05-26: 193
2007-05-27: 113
2007-05-28: 166
2007-05-29: 117
2007-05-30: 192
2007-05-31: 295
2007-06-01: 153
2007-06-02: 207
2007-06-03: 123
2007-06-04: 147
2007-06-05: 265
2007-06-06: 169
2007-06-07: 288
2007-06-08: 181
2007-06-09: 122
2007-06-10: 160
2007-06-11: 152
2007-06-12: 222
2007-06-13: 124
2007-06-14: 149
2007-06-15: 145
2007-06-16: 130
2007-06-17: 118
2007-06-18: 177
2007-06-19: 121
2007-06-20: 220
2007-06-21: 144
2007-06-22: 271
2007-06-23: 388
2007-06-24: 165
2007-06-25: 244
2007-06-26: 168
2007-06-27: 190
2007-06-28: 232
2007-06-29: 273
2007-06-30: 274
2007-07-01: 174
2007-07-02: 179
2007-07-03: 256
2007-07-04: 475
2007-07-05: 447
2007-07-06: 252
2007-07-07: 245
2007-07-0

2008-10-15: 37
2008-10-16: 31
2008-10-17: 25
2008-10-18: 3
2008-10-19: 8
2008-10-20: 40
2008-10-21: 63
2008-10-22: 66
2008-10-23: 9
2008-10-24: 33
2008-10-25: 31
2008-10-26: 55
2008-10-27: 26
2008-10-28: 26
2008-10-29: 20
2008-10-30: 19
2008-10-31: 32
2008-11-01: 39
2008-11-02: 43
2008-11-03: 17
2008-11-04: 34
2008-11-05: 18
2008-11-06: 13
2008-11-07: 48
2008-11-08: 32
2008-11-09: 21
2008-11-10: 132
2008-11-11: 62
2008-11-12: 11
2008-11-13: 30
2008-11-14: 12
2008-11-15: 40
2008-11-16: 16
2008-11-17: 33
2008-11-18: 18
2008-11-19: 39
2008-11-20: 206
2008-11-21: 36
2008-11-22: 15
2008-11-23: 19
2008-11-24: 31
2008-11-25: 33
2008-11-26: 92
2008-11-27: 38
2008-11-28: 27
2008-11-29: 31
2008-11-30: 38
2008-12-01: 26
2008-12-02: 115
2008-12-03: 74
2008-12-04: 19
2008-12-05: 23
2008-12-06: 29
2008-12-07: 35
2008-12-08: 13
2008-12-09: 22
2008-12-10: 50
2008-12-11: 34
2008-12-12: 37
2008-12-13: 118
2008-12-14: 68
2008-12-15: 22
2008-12-16: 110
2008-12-17: 34
2008-12-18: 65
2008-12-19: 44
2008-12-

...

2011-09-08: 87
2011-09-09: 143
2011-09-10: 95
2011-09-11: 114
2011-09-12: 80
2011-09-13: 64
2011-09-14: 95
2011-09-15: 89
2011-09-16: 31
2011-09-17: 57
2011-09-18: 67
2011-09-19: 70
2011-09-20: 52
2011-09-21: 42
2011-09-22: 64
2011-09-23: 28
2011-09-24: 56
2011-09-25: 43
2011-09-26: 37
2011-09-27: 16
2011-09-28: 56
2011-09-29: 30
2011-09-30: 20
2011-10-01: 82
2011-10-02: 8
2011-10-03: 22
2011-10-04: 19
2011-10-05: 6
2011-10-06: 15
2011-10-07: 12
2011-10-08: 33
2011-10-09: 16
2011-10-10: 53
2011-10-11: 41
2011-10-12: 21
2011-10-13: 33
2011-10-14: 32
2011-10-15: 29
2011-10-16: 12
2011-10-17: 38
2011-10-18: 4
2011-10-19: 73
2011-10-20: 17
2011-10-21: 29
2011-10-22: 14
2011-10-23: 9
2011-10-24: 13
2011-10-25: 7
2011-10-26: 31
2011-10-27: 45
2011-10-28: 71
2011-10-29: 15
2011-10-30: 13
2011-10-31: 13
2011-11-01: 7
2011-11-02: 180
2011-11-03: 18
2011-11-04: 21
2011-11-05: 8
2011-11-06: 34
2011-11-07: 48
2011-11-08: 114
2011-11-09: 66
2011-11-10: 92
2011-11-11: 41
2011-11-12: 27
2011-11-13: 5

2013-02-23: 40
2013-02-24: 31
2013-02-25: 20
2013-02-26: 14
2013-02-27: 41
2013-02-28: 6
2013-03-01: 32
2013-03-02: 24
2013-03-03: 55
2013-03-04: 26
2013-03-05: 20
2013-03-06: 47
2013-03-07: 22
2013-03-08: 18
2013-03-09: 28
2013-03-10: 25
2013-03-11: 21
2013-03-12: 70
2013-03-13: 30
2013-03-14: 54
2013-03-15: 30
2013-03-16: 29
2013-03-17: 25
2013-03-18: 30
2013-03-19: 50
2013-03-20: 60
2013-03-21: 60
2013-03-22: 85
2013-03-23: 93
2013-03-24: 53
2013-03-25: 96
2013-03-26: 59
2013-03-27: 61
2013-03-28: 55
2013-03-29: 107
2013-03-30: 60
2013-03-31: 83
2013-04-01: 87
2013-04-02: 112
2013-04-03: 194
2013-04-04: 117
2013-04-05: 157
2013-04-06: 94
2013-04-07: 92
2013-04-08: 130
2013-04-09: 103
2013-04-10: 108
2013-04-11: 198
2013-04-12: 158
2013-04-13: 202
2013-04-14: 106
2013-04-15: 163
2013-04-16: 149
2013-04-17: 185
2013-04-18: 139
2013-04-19: 175
2013-04-20: 108
2013-04-21: 147
2013-04-22: 141
2013-04-23: 244
2013-04-24: 185
2013-04-25: 234
2013-04-26: 285
2013-04-27: 308
2013-04-28: 207


2014-08-03: 190
2014-08-04: 462
2014-08-05: 295
2014-08-06: 239
2014-08-07: 265
2014-08-08: 362
2014-08-09: 235
2014-08-10: 178
2014-08-11: 206
2014-08-12: 207
2014-08-13: 204
2014-08-14: 208
2014-08-15: 206
2014-08-16: 223
2014-08-17: 150
2014-08-18: 140
2014-08-19: 210
2014-08-20: 211
2014-08-21: 108
2014-08-22: 213
2014-08-23: 171
2014-08-24: 156
2014-08-25: 169
2014-08-26: 160
2014-08-27: 166
2014-08-28: 322
2014-08-29: 205
2014-08-30: 156
2014-08-31: 135
2014-09-01: 146
2014-09-02: 155
2014-09-03: 127
2014-09-04: 137
2014-09-05: 110
2014-09-06: 112
2014-09-07: 96
2014-09-08: 76
2014-09-09: 83
2014-09-10: 55
2014-09-11: 80
2014-09-12: 75
2014-09-13: 39
2014-09-14: 93
2014-09-15: 60
2014-09-16: 62
2014-09-17: 53
2014-09-18: 45
2014-09-19: 74
2014-09-20: 51
2014-09-21: 50
2014-09-22: 52
2014-09-23: 67
2014-09-24: 32
2014-09-25: 67
2014-09-26: 45
2014-09-27: 15
2014-09-28: 55
2014-09-29: 31
2014-09-30: 36
2014-10-01: 49
2014-10-02: 20
2014-10-03: 23
2014-10-04: 86
2014-10-05: 29
2014-

2016-01-15: 106
2016-01-16: 62
2016-01-17: 24
2016-01-18: 120
2016-01-19: 25
2016-01-20: 74
2016-01-21: 56
2016-01-22: 94
2016-01-23: 111
2016-01-24: 52
2016-01-25: 40
2016-01-26: 105
2016-01-27: 73
2016-01-28: 57
2016-01-29: 95
2016-01-30: 79
2016-01-31: 26
2016-02-01: 43
2016-02-02: 38
2016-02-03: 28
2016-02-04: 42
2016-02-05: 23
2016-02-06: 38
2016-02-07: 54
2016-02-08: 84
2016-02-09: 24
2016-02-10: 44
2016-02-11: 17
2016-02-12: 34
2016-02-13: 16
2016-02-14: 10
2016-02-15: 69
2016-02-16: 60
2016-02-17: 164
2016-02-18: 29
2016-02-19: 13
2016-02-20: 22
2016-02-21: 36
2016-02-22: 32
2016-02-23: 11
2016-02-24: 17
2016-02-25: 23
2016-02-26: 36
2016-02-27: 23
2016-02-28: 62
2016-02-29: 9
2016-03-01: 13
2016-03-02: 16
2016-03-03: 10
2016-03-04: 35
2016-03-05: 24
2016-03-06: 16
2016-03-07: 48
2016-03-08: 26
2016-03-09: 4426
2016-03-10: 39
2016-03-11: 40
2016-03-12: 73
2016-03-13: 56
2016-03-14: 71
2016-03-15: 36
2016-03-16: 34
2016-03-17: 46
2016-03-18: 33
2016-03-19: 66
2016-03-20: 113
201

2017-06-26: 186
2017-06-27: 133
2017-06-28: 290
2017-06-29: 129
2017-06-30: 137
2017-07-01: 154
2017-07-02: 222
2017-07-03: 224
2017-07-04: 201
2017-07-05: 206
2017-07-06: 215
2017-07-07: 167
2017-07-08: 227
2017-07-09: 166
2017-07-10: 260
2017-07-11: 163
2017-07-12: 307
2017-07-13: 192
2017-07-14: 625
2017-07-15: 298
2017-07-16: 191
2017-07-17: 145
2017-07-18: 254
2017-07-19: 140
2017-07-20: 361
2017-07-21: 318
2017-07-22: 277
2017-07-23: 169
2017-07-24: 489
2017-07-25: 177
2017-07-26: 238
2017-07-27: 218
2017-07-28: 181
2017-07-29: 143
2017-07-30: 144
2017-07-31: 200
2017-08-01: 208
2017-08-02: 131
2017-08-03: 112
2017-08-04: 152
2017-08-05: 166
2017-08-06: 214
2017-08-07: 147
2017-08-08: 205
2017-08-09: 197
2017-08-10: 214
2017-08-11: 213
2017-08-12: 163
2017-08-13: 140
2017-08-14: 150
2017-08-15: 149
2017-08-16: 144
2017-08-17: 191
2017-08-18: 211
2017-08-19: 173
2017-08-20: 183
2017-08-21: 3660
2017-08-22: 196
2017-08-23: 187
2017-08-24: 184
2017-08-25: 198
2017-08-26: 159
2017-08

In [7]:
ordered_missing_pixels = sorted(results_dict.items(), key=lambda x:x[1])

for date, num_pixels in reversed(ordered_missing_pixels):
    print("{}: {}".format(date, num_pixels))

2005-10-15: 30262547
2005-10-22: 29790238
2005-10-21: 29783492
2005-10-23: 29656284
2005-10-24: 29637519
2005-10-25: 29536325
2005-10-27: 29422002
2005-10-28: 29338073
2005-10-29: 29331460
2005-10-31: 29212388
2005-10-30: 29207991
2005-12-30: 27515805
2005-12-27: 27498285
2005-12-31: 27495070
2005-12-29: 27489602
2005-12-28: 27451069
2006-04-21: 26833263
2006-09-22: 29955
2010-01-15: 21513
2013-05-09: 18328
2006-03-29: 16629
2005-10-03: 13969
2005-04-08: 9661
2017-02-26: 8605
2009-01-26: 6052
2010-07-11: 4677
2016-03-09: 4426
2017-08-21: 3660
2009-07-22: 3569
2016-09-01: 3216
2011-01-04: 2710
2013-11-03: 2478
2014-04-29: 2460
2014-10-23: 2441
2012-05-20: 2405
2004-04-19: 1819
2008-08-01: 1457
2008-02-07: 1161
2013-05-10: 963
2018-06-18: 727
2016-07-22: 693
2017-07-14: 625
2015-06-28: 616
2016-06-17: 607
2014-05-27: 567
2015-06-29: 565
2014-07-27: 549
2009-06-27: 547
2004-10-14: 543
2016-06-15: 530
2003-02-15: 522
2016-06-26: 521
2014-06-25: 515
2003-06-19: 498
2018-06-03: 496
2009-06-2

2016-09-07: 134
2014-04-19: 134
2011-07-25: 134
2011-07-24: 134
2008-05-03: 134
2006-08-29: 134
2006-07-31: 134
2005-06-05: 134
2003-07-24: 134
2003-04-18: 134
2018-04-07: 133
2017-06-27: 133
2017-04-14: 133
2016-07-26: 133
2015-12-01: 133
2015-08-14: 133
2015-04-12: 133
2013-08-23: 133
2009-04-10: 133
2008-05-14: 133
2006-07-07: 133
2006-04-16: 133
2005-09-07: 133
2005-07-28: 133
2005-04-13: 133
2004-04-29: 133
2016-11-16: 132
2015-09-02: 132
2015-08-29: 132
2013-09-06: 132
2012-09-05: 132
2011-01-13: 132
2010-04-19: 132
2009-04-20: 132
2008-12-28: 132
2008-11-10: 132
2008-04-20: 132
2008-04-17: 132
2006-08-25: 132
2006-08-24: 132
2004-11-03: 132
2004-08-26: 132
2004-08-09: 132
2004-07-21: 132
2004-04-23: 132
2004-04-05: 132
2003-06-23: 132
2003-04-14: 132
2017-08-02: 131
2017-02-01: 131
2016-08-24: 131
2016-01-08: 131
2015-04-01: 131
2013-05-04: 131
2012-09-03: 131
2012-05-09: 131
2011-09-05: 131
2011-09-03: 131
2011-07-04: 131
2009-07-09: 131
2009-06-02: 131
2009-01-12: 131
2008-01-

2011-10-11: 41
2009-11-14: 41
2007-11-19: 41
2007-03-18: 41
2007-02-04: 41
2007-02-01: 41
2006-12-24: 41
2006-11-20: 41
2006-11-07: 41
2005-12-24: 41
2004-11-19: 41
2004-03-21: 41
2018-03-25: 40
2017-10-28: 40
2016-12-31: 40
2016-09-18: 40
2016-03-11: 40
2016-01-25: 40
2015-12-19: 40
2015-02-01: 40
2014-12-14: 40
2014-12-12: 40
2014-11-27: 40
2014-11-17: 40
2014-11-10: 40
2014-11-07: 40
2014-10-16: 40
2014-03-16: 40
2014-02-03: 40
2013-11-28: 40
2013-10-28: 40
2013-02-23: 40
2012-11-14: 40
2012-03-24: 40
2012-03-18: 40
2011-12-25: 40
2011-01-01: 40
2010-11-16: 40
2010-10-05: 40
2008-11-15: 40
2008-10-20: 40
2008-03-24: 40
2008-03-22: 40
2006-01-06: 40
2005-11-05: 40
2005-03-19: 40
2004-03-10: 40
2004-02-17: 40
2004-02-02: 40
2004-01-05: 40
2003-09-18: 40
2003-03-23: 40
2017-02-19: 39
2016-03-10: 39
2015-03-27: 39
2015-01-18: 39
2014-12-06: 39
2014-11-01: 39
2014-10-15: 39
2014-09-13: 39
2013-12-09: 39
2013-01-24: 39
2012-11-18: 39
2012-03-09: 39
2012-01-12: 39
2011-12-05: 39
2011-03-18

In [ ]:
results_dict.values()
# plt.hist(results_dict.values(), bins=300000)
# plt.xlabel('Video duration (in seconds)')
# plt.ylabel('Number of videos')
# plt.title('Distribution of Video Durations')
# plt.axis([0, 20000, 0, 6000])
# plt.show()

In [ ]:
#         # Grayscale the image
#         imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#         # Threshold the image to black and white
#         _, im_thresh = cv2.threshold(imgray, 0, 255, cv2.THRESH_BINARY_INV)

#         # Copy the thresholded image
#         im_floodfill = im_thresh.copy()

#         # Fill in very small holes
#         small_kernel = np.ones((4,4),np.uint8)
#         im_floodfill = cv2.erode(im_floodfill, small_kernel, iterations = 1)

#         # Expand larger holes
#         large_kernel = np.ones((30,30),np.uint8)
#         im_floodfill = cv2.dilate(im_floodfill, large_kernel, iterations = 1)

#         # Mask used to flood filling.
#         # Notice the size needs to be 2 pixels than the image.
#         h, w = im_floodfill.shape
#         mask = np.zeros((h+2, w+2), np.uint8)

#         # Floodfill from top and bottom
#         for w_idx in range(w):
#             cv2.floodFill(im_floodfill, mask, (w_idx,0), 0);   # flood fill top
#             cv2.floodFill(im_floodfill, mask, (w_idx,h-1), 0); # flood fill bottom

#         # Count the number of holes
#         _, nr_objects = ndimage.label(im_floodfill)
#         pre_ratio_missing = 1.0 - (im_thresh == 0).sum() / im_thresh.size
#         post_ratio_missing = 1.0 - (im_floodfill == 0).sum() / im_thresh.size
#         missing_data_flag = False
#         if nr_objects == 0 and pre_ratio_missing > 0.165:
#             missing_data_flag = True
#             print("{}: Detected LARGE hole ({} missing data)".format(datestring, pre_ratio_missing))
#         if nr_objects != 0:
#             missing_data_flag = True
#             print("{}: Detected SMALL hole ({} missing data holes)".format(datestring, nr_objects))
        
#         if label == 0 and missing_data_flag: # False Positive
#             print("^^^ Algorithm INCORRECTLY detected missing data (FP)")
#         elif label == 1 and not missing_data_flag: # True Negative
#             print("{}: Algorithm MISSED the hole detection (TN).".format(datestring))